In [ ]:
from dask.distributed import Client
import dask
import numpy as np
import dask.array as da
from scipy import stats as sts
import random
from dask.distributed import progress

In [ ]:
# client = Client(n_workers=15)
client = Client(n_workers=8)
client

In [ ]:
%%time
n_vars = 16
n_times = 300
n_ens = 120

In [ ]:
arr_size = (n_ens, n_vars, n_times)
dx_1 = da.random.normal(0, 2, size=arr_size, chunks=(n_ens, n_vars, n_times // 100))
dx_2 = dx_1 + da.random.normal(0.3, 2.1, chunks=(n_ens, n_vars, n_times // 100))

In [ ]:
dx_1

In [ ]:
dx_1np = np.array(dx_1)
dx_2np = np.array(dx_2)

In [ ]:
@dask.delayed
def ks_rand_sel(data_1, data_2, idx_1, idx_2):
    return da.array(
        [
            [
                sts.ks_2samp(data_1[idx_1, _vix, _tix], data_2[idx_2, _vix, _tix])
                for _vix in range(data_1.shape[1])
            ]
            for _tix in range(data_1.shape[2])
        ]
    )

In [ ]:
def ks_rand_sel_np(data_1, data_2, idx_1, idx_2):
    return np.array(
        [
            [
                sts.ks_2samp(data_1[idx_1, _vix, _tix], data_2[idx_2, _vix, _tix])
                for _vix in range(data_1.shape[1])
            ]
            for _tix in range(data_1.shape[2])
        ]
    )

In [ ]:
dx_1 = dx_1.rechunk({0: "auto", 1: "auto", 2: "auto"}, block_size_limit=1e8)
dx_2 = dx_2.rechunk({0: "auto", 1: "auto", 2: "auto"}, block_size_limit=1e8)
dx_1

In [ ]:
%%time
random.seed(101_114)
futures = []
test_size = 30
for rse in range(10):
    idx_1 = random.sample(list([_ for _ in range(dx_1.shape[0])]), test_size)
    idx_2 = random.sample(list([_ for _ in range(dx_2.shape[0])]), test_size)
    futures.append(client.submit(ks_rand_sel, dx_1, dx_2, idx_1, idx_2))
results = dask.compute(*client.gather(futures))
progress(futures)

In [ ]:
%%time
random.seed(101_114)
futures = []
test_size = 30
for rse in range(10):
    idx_1 = random.sample(list([_ for _ in range(dx_1np.shape[0])]), test_size)
    idx_2 = random.sample(list([_ for _ in range(dx_2np.shape[0])]), test_size)
    futures.append(ks_rand_sel_np(dx_1np, dx_2np, idx_1, idx_2))

In [ ]:
results = da.array(results)

In [ ]:
results.var().compute()